## 주의

이 tutorials 는 큰 용량의 메모리를 요구합니다. 충분한 메모리가 확보된 각자의 환경에서만 테스트 해보시길 권장합니다. 

Scikit-learn 의 logistic regression 을 이용하기 위하여 데이터를 한 번에 메모리에 올리기 때문인데, 대략 20 GB 를 이용합니다.

이 부분은 PyTorch 나 TensorFlow 로 훨씬 효율적으로 구현할 수 있습니다. 이후에 효율적인 구현체를 추가로 올리겠습니다.

## Load dataset

Comments 는 list of str 형식의 문장을 yield 하는 데이터클래스 입니다. 영화평 데이터는 (영화 id, 영화평, 평점) 의 column data 이기 때문에 영화평만 선택하여 띄어쓰기 기준으로 split 합니다.

In [1]:
import config
from navermovie_comments import get_movie_comments_path

class Comments:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for i, doc in enumerate(f):
                idx, text, rate = doc.split('\t')
                yield text.split()

path = get_movie_comments_path(large=True, tokenize='soynlp_unsup')
comments = Comments(path)

for i, sent in enumerate(comments):
    if i >= 3:
        break
    print(sent)

soynlp=0.0.49
added lovit_textmining_dataset
['명불허전']
['왠지', '고사', '피의', '중간', '고사', '보다', '재미', '가', '없을듯', '해요', '만약', '보게', '된다면', '실망', '할듯']
['티아라', '사랑', '해', 'ㅜ']


## Scan vocabulary

데이터에 존재하는 단어들을 `min_count` 기준으로 필터링합니다. Return 값으로 vocabulary index 와 count 를 return 합니다.

In [2]:
from collections import defaultdict

def scan_vocabulary(sents, min_count, verbose=False):
    counter = defaultdict(int)
    for i, sent in enumerate(sents):
        if verbose and i % 100000 == 0:
            print('\rscanning vocabulary .. from %d sents' % i, end='')
        for word in sent:
            counter[word] += 1
    counter = {word:count for word, count in counter.items()
               if count >= min_count}
    idx_to_vocab = [vocab for vocab in sorted(counter,
                    key=lambda x:-counter[x])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    idx_to_count = [counter[vocab] for vocab in idx_to_vocab]
    if verbose:
        print('\rscanning vocabulary was done. %d terms from %d sents' % (len(idx_to_vocab), i+1))
    return vocab_to_idx, idx_to_vocab, idx_to_count

vocab_to_idx, idx_to_vocab, idx_to_count = scan_vocabulary(
    comments, min_count=10, verbose=True)

scanning vocabulary was done. 69541 terms from 3280685 sents


scan_vocabulary 의 return 값을 살펴봅니다.

In [3]:
print(idx_to_vocab[:5])
print(idx_to_count[:5])

['영화', '이', '관람', '객', '의']
[1128809, 866305, 600351, 526070, 489950]


## Features

우리는 window = 2 를 이용하여 한 단어 X[0] 의 앞, 뒤 각각 두 개의 단어 (총 4개의 단어)를 X[0] 의 features 로 이용합니다. 예를 들어 (a, b, c, d, e) 라는 단어가 등장하였고, X[0]=c 라면 X[-2]=a, X[-1]=b, X[1]=d, X[2]=e 입니다.

그리고 scan vocabulary 함수를 통하여 학습된 단어가 총 5 개라면 이들의 위치를 보존하면 X[0] 에 대한 feature space 를 20 만 차원으로 만들 수 있습니다. 만약 각 단어의 index 가 {a:0, b:1, c:2, d:3, e:4} 라면 X[0]=c 는 [0, 5+1, 10+3, 15+4] 를 features 로 가진다 표현할 수 있습니다.

feature_to_idx 는 이를 만드는 함수입니다. 문장 내에서 현재 단어 X[0] 의 위치를 i, 현재 단어 앞, 뒤 단어인 X[-1] 이나 X[1] 의 위치를 j, j 위치의 단어의 index 를 vocab_idx 라 할 때, 이 값의 feature index 를 출력합니다.

In [4]:
def feature_to_idx(i, j, vocab_idx, window, n_terms):
    if j < i:
        return n_terms * (j - i + window) + vocab_idx
    else:
        return n_terms * (j - i + window - 1) + vocab_idx

feature_to_idx(i=2, j=3, vocab_idx=3, window=2, n_terms=5)

13

idx_to_feature 는 반대로 feature index 를 feature 로 decode 합니다. feature idx 를 vocabulary 의 개수로 나눈 몫은 상대적 위치값이 되고, 나머지는 vocabulary idx 입니다.

In [5]:
def idx_to_feature(feature_idx, idx_to_vocab, window):
    # 몫
    position = feature_idx // len(idx_to_vocab)
    if position < window:
        feature = 'X[-%d] = ' % (window - position)
    else:
        feature = 'X[%d] = ' % (position - window + 1)
    # 나머지
    vocab_idx = feature_idx % len(idx_to_vocab)
    feature += idx_to_vocab[vocab_idx]
    return feature

idx_to_feature(13, 'a b c d e'.split(), window=2)

'X[1] = d'

이를 이용하여 학습데이터로부터 window classification 용 데이터를 만듭니다. X[0] 을 기준으로 X[-2], X[-1], X[1], X[2] 의 co-occurrence 를 계산하는 matrix 를 만듭니다. Sparse matrix 형식이기 때문에 rows, columns 를 따로 모읍니다. words 는 각 rows 에 해당하는 단어를 넣어둡니다.

아래 부분에서 scan vocabulary 의 결과에 포함되지 않은 단어는 건너 띄며, context words 의 범위는 문장의 맨 앞에서 문장의 맨 뒷 단어가 되도록 index 의 범위를 확인합니다.

```python
for i, word in enumerate(sent):
    if not (word in vocab_to_idx):
        continue

    b = max(0, i - window)
    e = min(i + window, n_words)
```

아래 구문을 통하여 sent[j] 의 단어 역시 scan vocabulary 의 결과에 포함되지 않으면 이를 이용하지 않습니다.

```python
for j in range(b, e):
    if i == j:
        continue
    j_idx = vocab_to_idx.get(sent[j], -1)
    if j_idx == -1:
        continue
```

In [6]:
import numpy as np
from scipy.sparse import csr_matrix

def create_window_cooccurrence_matrix(vocab_to_idx, sentences, window=2, verbose=True):

    n_terms = len(vocab_to_idx)

    rows = []
    cols = []
    words = []

    row_idx = 0
    col_idx = window * 2 * n_terms

    for i_sent, sent in enumerate(sentences):

        if verbose and i_sent % 10000 == 0:
            print('\rcreating train dataset {} rows from {} sents'.format(row_idx, i_sent), end='')

        n_words = len(sent)

        for i, word in enumerate(sent):
            if not (word in vocab_to_idx):
                continue

            b = max(0, i - window)
            e = min(i + window, n_words)

            features = []
            for j in range(b, e):
                if i == j:
                    continue
                j_idx = vocab_to_idx.get(sent[j], -1)
                if j_idx == -1:
                    continue
                features.append(feature_to_idx(i, j, j_idx, window, n_terms))

            if not features:
                continue

            # sparse matrix element
            for col in features:
                rows.append(row_idx)
                cols.append(col)

            # words element
            words.append(word)

            row_idx += 1

    if verbose:
        print('\rtrain dataset {} rows from {} sents was created    '.format(row_idx, i_sent))

    # to csr matrix
    rows = np.asarray(rows, dtype=np.int)
    cols = np.asarray(cols, dtype=np.int)
    data = np.ones(rows.shape[0], dtype=np.int)
    X = csr_matrix((data, (rows, cols)), shape=(row_idx, col_idx))

    return X, words

In [7]:
window = 2

X, words = create_window_cooccurrence_matrix(
    vocab_to_idx, comments, window)

train dataset 42981576 rows from 3280684 sents was created    


## Word2Vec 을 이용한 seed set 만들기

미리 학습해둔 Word2Vec model 을 로딩합니다. 우리는 사람 이름을 인식하는 named entity recognizer 를 만들겁니다. `송강호`와 `디카프리오`의 Word2Vec 유사어는 사람 이름임을 알 수 있습니다. 각각 100 개씩의 유사어를 선택하여 이의 합집합을 seed_words 로 선택합니다. 총 172 개의 단어가 seeds 로 선택되었습니다.

In [8]:
from pprint import pprint
from navermovie_comments import load_trained_embedding


word2vec = load_trained_embedding(tokenize='soynlp_unsup')

pprint(word2vec.wv.most_similar('송강호', topn=10))
pprint(word2vec.wv.most_similar('디카프리오', topn=10))

seed_words = {word for word, _ in word2vec.most_similar('송강호', topn=100)}
seed_words.update({word for word, _ in word2vec.most_similar('디카프리오', topn=100)})

len(seed_words)

[('하정우', 0.9089709520339966),
 ('한석규', 0.882610023021698),
 ('오달수', 0.8565834760665894),
 ('이정재', 0.8554919958114624),
 ('김명민', 0.8462209105491638),
 ('이범수', 0.8421300649642944),
 ('설경구', 0.8421212434768677),
 ('황정민', 0.8380463719367981),
 ('손현주', 0.8376869559288025),
 ('김윤석', 0.8338449597358704)]
[('레오', 0.8400799036026001),
 ('톰하디', 0.8309459686279297),
 ('앤해서웨이', 0.775414764881134),
 ('앤헤서웨이', 0.7646037340164185),
 ('브래드피트', 0.7506041526794434),
 ('로다주', 0.7499849796295166),
 ('로버트드니로', 0.7493538856506348),
 ('콜린퍼스', 0.737823486328125),
 ('히스레저', 0.7330209016799927),
 ('윌스미스', 0.7309495806694031)]


/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


172

In [14]:
pprint(word2vec.wv.most_similar('송강호', topn=100))
pprint(word2vec.wv.most_similar('디카프리오', topn=100))

[('하정우', 0.9089709520339966),
 ('한석규', 0.882610023021698),
 ('오달수', 0.8565834760665894),
 ('이정재', 0.8554919958114624),
 ('김명민', 0.8462209105491638),
 ('이범수', 0.8421300649642944),
 ('설경구', 0.8421212434768677),
 ('황정민', 0.8380463719367981),
 ('손현주', 0.8376869559288025),
 ('김윤석', 0.8338449597358704),
 ('유해진', 0.8303444385528564),
 ('주진모', 0.828080952167511),
 ('공유', 0.8169301748275757),
 ('이병헌', 0.8125848770141602),
 ('문정희', 0.8114207983016968),
 ('정우', 0.8097195029258728),
 ('최민식', 0.8086514472961426),
 ('안성기', 0.8084512948989868),
 ('김윤진', 0.805782675743103),
 ('성동일', 0.7982152700424194),
 ('조진웅', 0.7979434728622437),
 ('조정석', 0.7975532412528992),
 ('안성기씨', 0.7943410873413086),
 ('류승룡', 0.7933757901191711),
 ('정재영씨', 0.790336012840271),
 ('진구', 0.7890698909759521),
 ('손예진', 0.7869455814361572),
 ('이하늬', 0.7824959754943848),
 ('이제훈', 0.7823973894119263),
 ('감우성', 0.7821334600448608),
 ('정재영', 0.7818644046783447),
 ('박신양', 0.7780541181564331),
 ('고수', 0.7775152921676636),
 ('윌스미스', 0.7771

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Word2Vec 유사어를 이용하여 label vector 만들기

앞서 만든 X 의 row 에 해당하는 단어가 seed_words 에 포함될 경우, 이 rows 의 값을 1 로, 그렇지 않은 경우 0 으로 지정합니다.

172 개의 단어가 361,394 번 등장하였습니다.

In [9]:
y = np.zeros(X.shape[0], dtype=np.int)
for i, word in enumerate(words):
    if word in seed_words:
        y[i] = 1

y.sum()

361394

## Logistic Regression 을 이용한 window classifier 만들기

Logistic regression 을 학습합니다. seed words 를 positive class 로 예측하는 모델을 만듭니다. 그런데 실제로는 사람 이름이면서도 label 을 0 으로 가지는 데이터도 존재합니다. Logistic regression 은 이들에 대해서는 큰 확률값을 지닐 가능성이 높습니다. Training error 를 named entity 의 힌트로 이용하는 것입니다. 이 때의 training error 는 우리가 seed words 를 이용하여 엉성하게 데이터를 준비했기 때문에 발생하는 error 이기 때문입니다.

In [11]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.fit(X, y)
y_pred = logistic.predict(X)
y_prob = logistic.predict_proba(X)[:,1]

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


모델은 softmax probability 가 0 에 가까운 값이면 일단은 negative class 로 분류하여 loss 가 작을 것입니다. 그리고 사람이 등장하는 문맥에서 등장하는 negative class 의 softmax probability 를 지나치게 줄이려 하면 positive class 의 확률값이 매우 작게 되기 때문에 negative class 이면서 사람인 단어들에 대해서는 0 보다는 크되, 매우 작은 확률을 부여합니다. 이 점을 이용하여 prediction probability 가 0.05 보다 큰 snapshot (row) 들을 `pred_pos` 에 카운팅 합니다.

이후 해당 단어가 등장한 횟수로 0.05 보다 큰 prediction probability 를 받은 횟수를 나눠 named entity score 를 계산합니다.

예를 들어 배우 `백윤식` 은 seed words 에 포함되지 않았지만 총 100 번 등장하였고, 그 중 95 번을 0.05 보다 큰 prediction probability 를 받았다면, 이 단어의 named entity score 는 0.95 가 됩니다.

In [12]:
from collections import Counter

# word count
word_counter = Counter(words)

# prediction count
pred_pos = defaultdict(int)
for row in np.where(y_prob >= 0.05)[0]:
    pred_pos[words[row]] += 1
pred_pos = {word:pos/word_counter[word] for word, pos in pred_pos.items()}

## 결과 확인하기

Named entity score 가 큰 순서대로 상위 1000 개의 단어를 선택합니다. 그 중 seed words 에 포함된 단어는 출력하지 않습니다. (단어, 빈도수), score 를 확인합니다. 실제로 `백윤식` 은 293 번 등장했으며, 약 222 번 0.05 보다 큰 probability 를 얻었습니다.

In [13]:
for word, prob in sorted(pred_pos.items(), key=lambda x:-x[1])[:1000]:
    if word in seed_words:
        continue
    idx = vocab_to_idx[word]
    count = idx_to_count[idx]
    print('{} ({})\t{:.3f}'.format(word, count, prob))

해서워이 (10)	1.000
그브가 (12)	1.000
장현성 (11)	1.000
왕이고싶었고 (26)	0.962
공지영작가 (13)	0.923
신정근 (11)	0.909
달화 (10)	0.900
헤더웨이 (10)	0.900
전국환 (10)	0.900
헤서웨이 (261)	0.893
틸타 (10)	0.889
박원상 (23)	0.870
패틴슨 (286)	0.864
천의 (42)	0.857
와저 (188)	0.849
동해물 (46)	0.848
곽동원 (12)	0.833
류승수 (12)	0.833
레져 (78)	0.808
고슬링 (195)	0.805
참바다 (81)	0.802
김동욱씨 (10)	0.800
동명수 (15)	0.800
해써웨 (10)	0.800
진짫 (10)	0.800
헤스 (10)	0.800
김소담 (10)	0.800
마형 (15)	0.800
계두식 (10)	0.800
윤지혜 (26)	0.800
유이인 (15)	0.800
종석 (183)	0.796
하저우 (14)	0.786
임현식 (13)	0.769
전혜진씨 (13)	0.769
크루주 (17)	0.765
희순 (21)	0.762
백윤식 (293)	0.758
손현주아저씨 (37)	0.757
ㅋㅋ이민기 (80)	0.750
설경규 (12)	0.750
하장우 (12)	0.750
류성룡 (12)	0.750
혜정 (12)	0.750
톰크르즈 (16)	0.750
슈왈제너거 (16)	0.750
계룬미 (12)	0.750
엄태구씨 (39)	0.744
예진이 (36)	0.743
김남일 (31)	0.742
석규 (28)	0.741
패티슨 (27)	0.741
이성민 (452)	0.739
두연기자 (19)	0.737
이병현씨 (19)	0.737
쿠르즈 (19)	0.737
김윤식씨 (34)	0.735
현주 (15)	0.733
천호진 (30)	0.733
유승룡씨 (15)	0.733
히들스턴 (15)	0.733
스코티 (26)	0.731
시후 (37)	0.730
하지원언니 (44)	0.727
박종훈 (11)	0.727
영애 (11

## Named Entity Filter (Feature) 확인하기

우리는 window instance 를 하나의 row 로 만들었기 때문에 prediction probability 가 높은 instance 를 확인하면, 어떤 context 에서 X[0] 가 사람 이름인지를 확인할 수 있습니다.

In [15]:
top_instances = np.where(0.7 <= y_prob)[0]
top_probs = y_prob[top_instances]

print(top_instances.shape)
print(top_probs.shape)

(32775,)
(32775,)


32,775 개의 rows 중에는 중복된 것들도 많습니다. 중복된 경우를 정리하여 각 instance 와 count, 그리고 prediction probability 를 정리하는 함수를 만듭니다.

In [16]:
from collections import defaultdict

def get_unique_top_instances(sample_idxs, probs):

    # slice samples
    X_samples = X[sample_idxs]
    rows, cols = X_samples.nonzero()

    # find unique instance
    instance_prob = {}
    instance_count = defaultdict(int)
    before_row = None

    def update_dict(features, prob):
        features = sorted(features, key=lambda x:x[0])
        instance = ', '.join(f[1] for f in features)
        instance_prob[instance] = prob
        instance_count[instance] += 1
        return []

    features = [] # temporal variable
    for row, feature_idx in zip(rows, cols):
        # update unique dictionary
        if row != before_row and features:
            features = update_dict(features, probs[row])
        # update temporal variable
        before_row = row
        feature = idx_to_feature(feature_idx, idx_to_vocab, window)
        features.append((feature_idx, feature))

    # last elements
    if features:
        update_dict(features, probs[row])

    return instance_prob, instance_count

여진히 unique instance (unique input vectors) 의 개수가 많습니다.

In [17]:
instance_prob, instance_count = get_unique_top_instances(top_instances, top_probs)
len(instance_prob)

16611

빈도수 기준으로 상위 500 개의 instance 를 출력합니다 (probability, count), instance 입니다.

뒤에 X[1] = '씨' 가 등장하는 경우가 가장 많았으며, 아래쪽에는 다음과 같은 표현도 있습니다. `믿고보는 송강호` 와 같은 전형적인 영화평 도메인에서의 표현입니다.

```
(0.8705, 16)	X[-2] = 역시, X[-1] = 믿고보는, X[1] = 과
```

아래의 표현은 `아놀드`가 seed words 에 포함되었기 때문에 `슈왈제네거`라는 단어 앞의 단어를 사람 이름으로 인식한 경우입니다.

```
(0.9849, 71)	X[1] = 슈왈제네거
```

또한 영화평에서는 배우의 이름을 나열하는 경우들도 있습니다. 배우의 이름을 다 나열한 뒤, '이 캐스팅봐라' 식의 표현들이 있어서 배우 이름 역시 유의미한 context 로 선택됩니다.

```
(0.7966, 8)	X[-2] = 강동원, X[-1] = 황정민, X[1] = 이성민
```

In [18]:
sorted_instance_count = sorted(instance_count.items(), key=lambda x:-x[1])

for instance, count in sorted_instance_count[:500]:
    prob = instance_prob[instance]
    print('({:.4}, {})\t{}'.format(prob, count, instance))

(0.7244, 3394)	X[1] = 씨
(0.9009, 701)	X[1] = 씨의
(0.8886, 658)	X[-2] = 관람, X[-1] = 객, X[1] = 씨
(0.7233, 397)	X[1] = 씨가
(0.9071, 291)	X[-2] = 관람, X[-1] = 객, X[1] = 님
(0.7244, 182)	X[1] = 씨는
(0.7797, 172)	X[-2] = 관람, X[-1] = 객, X[1] = 씨의
(0.8618, 136)	X[1] = 레저
(0.909, 121)	X[1] = 씨와
(0.8513, 100)	X[-2] = 재밌, X[-1] = 어요, X[1] = 씨
(0.7012, 87)	X[-2] = 재밌, X[-1] = 었어요, X[1] = 씨
(0.7819, 76)	X[-2] = 관람, X[-1] = 객, X[1] = 님의
(0.7439, 75)	X[-2] = 게, X[-1] = 봤어요, X[1] = 씨
(0.7826, 73)	X[-2] = 잘, X[-1] = 봤습니다, X[1] = 씨
(0.9591, 73)	X[-2] = 관람, X[-1] = 객, X[1] = 씨가
(0.8305, 72)	X[-2] = 개인적, X[-1] = 으로, X[1] = 씨
(0.8472, 71)	X[1] = 퍼스
(0.9849, 71)	X[1] = 슈왈제네거
(0.7257, 69)	X[-2] = 타마코, X[-1] = 나의
(0.9148, 60)	X[-1] = 히스, X[1] = 의
(0.7965, 59)	X[1] = 씨랑
(0.9407, 58)	X[-1] = 믿고보는, X[1] = 배우
(0.7257, 57)	X[1] = 씨를
(0.7965, 56)	X[-2] = 게, X[-1] = 봤습니다, X[1] = 씨
(0.8211, 53)	X[-2] = 있, X[-1] = 었어요, X[1] = 씨
(0.7439, 50)	X[1] = 피트
(0.8999, 47)	X[-1] = 역시, X[1] = 님
(0.8373, 47)	X[-2] = 영화, X[-1] = 였습니다, 

우리는 sparse vector representation 형태의 input data 를 만들었기 때문에 logistic regression 을 이용했습니다. 이와 다르게 X[-2], X[-1], X[1], X[2] 의 단어의 word vector 를 concatenation 한다면 neural network 의 input 이 될 수 있습니다. 이러한 방식이 아래의 Stanford 수업의 assignment 내용입니다.

https://nlp.stanford.edu/~socherr/pa4_ner.pdf

또한 word vector 를 이용한다면 Convolutional Neural Network (CNN) 를 이용할 수도 있습니다. 특히나 CNN 은 n-gram features 를 이용하기 때문에 문맥 정보가 더 정확히 잡힐 수 있습니다. 이에 대해서는 다른 튜토리얼에서 다룹니다.